In [1]:
# %pip install torcheval
# %pip install openai-whisper

In [2]:
from torcheval.metrics import BLEUScore, Perplexity, WordErrorRate, WordInformationLost, WordInformationPreserved

In [3]:
bleu = BLEUScore(n_gram=4)
perplexity = Perplexity()
wer = WordErrorRate()
wil = WordInformationLost()
wilp = WordInformationPreserved()

In [4]:
def num_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
import whisper

whisper_base = whisper.load_model("base")
whisper_large = whisper.load_model("large")

In [15]:
import os
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import torchaudio
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB

datapath = r"D:\Database\s2t-yt\source"
def load_youtube_dataset(datapath, ext="mp3"):
    res = list()
    for root, dirs, files in os.walk(datapath):
        for vid in dirs:
            for root, dirs, files in os.walk(os.path.join(datapath, vid)):
                a = [os.path.join(root, f) for f in Path(root).rglob(f"*.{ext}")]
                t = [os.path.join(root, f) for f in Path(root).rglob("segment*.txt")]
                res.extend(list(zip(a, t)))
                # print(res)


    return res


data = load_youtube_dataset(datapath)

def load_data(data):
    res = list()
    for audio, text in data:        
        with open(text, "r", encoding="utf-8") as f:
            text = f.read().strip()
        res.append((audio, text))
    return res


data = load_data(data)

mel = MelSpectrogram()
amp = AmplitudeToDB()

def preprocess_audio(audio):
    audio = torchaudio.load(audio)[0]
    audio = mel(audio)
    audio = amp(audio)
    return audio

def preprocess_text(text):
    return text

def preprocess_data(data):
    audio, text = data
    audio = preprocess_audio(audio)
    text = preprocess_text(text)
    return audio, text

def preprocess_batch(batch):
    return list(map(preprocess_data, batch))

def preprocess_dataset(dataset):
    return list(map(preprocess_data, dataset))

# data = preprocess_dataset(data)

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        for audio, text in data:
            output = model.transcribe(audio)["text"]
            # print(output)
            bleu.update(output, [text])
            # perplexity.update(output, text)
            wer.update(output, text)
            wil.update(output, text)
            wilp.update(output, text)

print("Base: ", num_params(whisper_base))
evaluate(whisper_base, data)
print("Base BLEU: ", bleu.compute())
# print("Base Perplexity: ", perplexity.compute())
print("Base WER: ", wer.compute())
print("Base WIL: ", wil.compute())
print("Base WILP: ", wilp.compute())

print("Large: ", num_params(whisper_large))
evaluate(whisper_large, data)
print("Large BLEU: ", bleu.compute())
# print("Large Perplexity: ", perplexity.compute())
print("Large WER: ", wer.compute())
print("Large WIL: ", wil.compute())
print("Large WILP: ", wilp.compute())

d:\workspace\s2t\.nemo\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
d:\workspace\s2t\.nemo\lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Base:  71825920
 Wenn ich für meine Eltern spreche, dann mache ich meine Adoptiv-Eltern, mein Onkel, meine Tante, aber weil ich bei denen seit meinem Pünften leben sehe.
 Ich habe als Kind, vor allem auf einer Teenagerzeit immer mal wieder diese Gedanken.
 Wir haben uns verabschiedet und er hat mir gesagt, ich wünsche dich wäre du gewesen. Ich wünsche dich wäre in der Zeit. Und das ist schon natürlich dabei.
 sich sich zu sagen, das trauen sich den Menschen nicht.
 als die in bewegen auch anders, also in Kontext von meiner Autobahn, plus meine Behinderung. Und darauf hin, was kann ich auch so, dass ich dann zum Beispiel zu meiner Oma immer wieder gesagt habe.
 geblickt und auch auf mein Leben in Deutschland und während mich an alle Situationen und genera, als wir es letztendom all dann gesehen hatten. Wir im Flughafen standen und
 dafür gekämpft, dass die das Adoptionsrecht bekommen. Und das dauert ja an Deutschland nur ein bisschen und deswegen bin ich dann erst mit fünf Jahren dann a

ValueError: the input is too short to find all n-gram matches with n_gram=4

In [ ]:
datafor 